In [8]:
!pip install llama_cloud_services


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from financial_schemas_end import EndowmentAndInvestmentLevels_2024_25  #This could be adjusted through schemas.py
#from dotenv import load_dotenv


In [12]:
PDF_ROOT = "university_pdfs_test"
OUTPUT_ROOT = "output_fin_nk"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "3c22ff5d-0d8d-4aab-8e85-c9f62677f28b" #Different based on your LLamaCloud account
api_key = os.getenv("LLAMACLOUD_API_KEY")

In [28]:
extractor = LlamaExtract(
    api_key="llx-MEn3MyAlMLfoRFu6eUItoKZ2VPD5wt9mAwufvdypqq1n7XrL",  
    project_id="34179e7d-a43e-4c15-b703-536cff419e9b"
)



#agent = extractor.create_agent(name = "endowment-parser-2024", data_schema=EndowmentAndInvestmentLevels_2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = EndowmentAndInvestmentLevels_2024_25.schema()
agent.save()
agent = extractor.get_agent(id = AGENT_ID)


/var/folders/m1/8j3nd3m95y1brb0j52fx43fr0000gn/T/ipykernel_82704/2139312051.py:13: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent.data_schema = EndowmentAndInvestmentLevels_2024_25.schema()





Uploading files:   0%|                                    | 0/1 [00:00<?, ?it/s]




Uploading files: 100%|████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]





Creating extraction jobs:   0%|                           | 0/1 [00:00<?, ?it/s]




Creating extraction jobs: 100%|███████████████████| 1/1 [00:00<00:00,  2.46it/s]





Extracting files:   0%|                                   | 0/1 [00:00<?, ?it/s]




Extracting files: 100%|███████████████████████████| 1/1 [00:09<00:00,  9.22s/it]





Uploading files:   0%|                                    | 0/1 [00:00<?, ?it/s]




Uploading files: 100%

In [29]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'endowment_net_assets_eoy': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Total endowment net assets at the end of fiscal year 2024–2025. May be labeled as 'Endowment Net Assets', 'EOY Endowment', 'Total Endowment Assets', etc. Only extract if the amount clearly refers to end-of-year value. Ignore data from prior years or interim periods. Do not derive or assume this value unless clearly labeled in the document."},
  'appropriation_of_endowment_for_expenditure': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Appropriations from endowment for expenditure in 2024–2025. May appear as 'Spending from Endowment', 'Appropriation of Endowment Income', 'Endowment Draw', or similar. Only extract if this relates to the 2024–2025 period. Do not derive or infer this amount if not explicitly reported."},
  'investment_level_1': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "F

The following two cell blocks extract all schools' info into one excel file per school.

In [30]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [31]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

Processing school: ASU
Extracting data from P11817713-P11393130-P11833489.pdf
Saved output to output_fin_nk/ASU.xlsx
Processing school: Bradley University
Extracting data from P21862068-P21425154-P21869099.pdf
Saved output to output_fin_nk/Bradley University.xlsx
Processing school: California_state_university
Extracting data from P21878315-P21436983-P21882690.pdf
Saved output to output_fin_nk/California_state_university.xlsx
Processing school: Cornell_university
Extracting data from P11799657-P11380074-P11818843.pdf
Saved output to output_fin_nk/Cornell_university.xlsx
Processing school: Culinary_institute_of_America
Extracting data from P11790595-P11373821-P11811978.pdf
Saved output to output_fin_nk/Culinary_institute_of_America.xlsx
Processing school: Gannon_university
Extracting data from P21859160-P21423095-P21866877.pdf


Uploading files:   0%|                                    | 0/1 [02:02<?, ?it/s]


Saved output to output_fin_nk/Gannon_university.xlsx
Processing school: Harvard_university
Extracting data from P21889042-P21444694-P21891364.pdf
Saved output to output_fin_nk/Harvard_university.xlsx
Processing school: Lewis_univsersity
Extracting data from P11819634-P11394595-P11835096.pdf
Saved output to output_fin_nk/Lewis_univsersity.xlsx
Processing school: MT_ST_MARY
Extracting data from Mt St Mary's fall 24 continuing disclosure.pdf
Saved output to output_fin_nk/MT_ST_MARY.xlsx
Processing school: Michigan_state_university
Extracting data from P21870305-P21430806-P21875444.pdf
Saved output to output_fin_nk/Michigan_state_university.xlsx
Processing school: Molloy_college
Extracting data from P21874771-P21434198-P21879428.pdf
Saved output to output_fin_nk/Molloy_college.xlsx
Processing school: New_York_University
Extracting data from P11812334-P11389223-P11829036.pdf
Saved output to output_fin_nk/New_York_University.xlsx
Processing school: Ohio_state
Extracting data from P21875437-P

The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [32]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

Extracting data from ASU/P11817713-P11393130-P11833489.pdf
Extracting data from Bradley University/P21862068-P21425154-P21869099.pdf
Extracting data from California_state_university/P21878315-P21436983-P21882690.pdf
Extracting data from Cornell_university/P11799657-P11380074-P11818843.pdf
Extracting data from Culinary_institute_of_America/P11790595-P11373821-P11811978.pdf
Extracting data from Gannon_university/P21859160-P21423095-P21866877.pdf
Extracting data from Harvard_university/P21889042-P21444694-P21891364.pdf
Extracting data from Lewis_univsersity/P11819634-P11394595-P11835096.pdf
Extracting data from MT_ST_MARY/Mt St Mary's fall 24 continuing disclosure.pdf
Extracting data from Michigan_state_university/P21870305-P21430806-P21875444.pdf
Extracting data from Molloy_college/P21874771-P21434198-P21879428.pdf
Extracting data from New_York_University/P11812334-P11389223-P11829036.pdf
Extracting data from Ohio_state/P21875437-P21434721-P21880042.pdf
Extracting data from ST_Louis_univ

In [33]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_fin_nk/all_schools.xlsx"
output_path = "output_fin_nk/all_schools_combined.xlsx"

raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", "2024‑2025")

#df_comb.loc['Texas_A&M', ['Total_Headcount','Undergraduate_Headcount']] = \
    #df_comb.loc['Texas_A&M', ['Undergraduate_Headcount','Total_Headcount']].values

# df_comb.loc['California_state_university', 'Undergraduate_Headcount'] = None 
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)

Saved: output_fin_nk/all_schools_combined.xlsx
